# [데이터타입통합]
### kopo_channel_seasonality_new.csv 파일을 불러온 후 selloutData 변수에 담으세요. 이후 QTY컬럼→실수(float), 이외컬럼→문자(str)로변경하세요

In [1]:
### 1. 라이브러리 선언하기

import pandas as pd
import numpy as np

### 2. 데이터 불러오기

selloutData = \
    pd.read_csv("../dataset/kopo_channel_seasonality_new.csv",encoding = "ms949")

selloutData.describe()   # describe -> 숫자기본 통계량보여줌??  음수 : 

selloutData.dtypes

selloutData["QTY"] = selloutData["QTY"].astype(float)

selloutData["YEARWEEK"] = selloutData["YEARWEEK"].astype(str)

selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK     object
QTY         float64
dtype: object

In [ ]:
####여기서 부터 실습~!~!

# 3-1.[불량데이터처리]
#### kopo_channel_seasonality_new.csv 자료를담은selloutData 변수에서 QTY컬럼음수(반품)인경우0, 양수인경우기존QTY 값유지하는로직을 적용하여QTY_NEW 컬럼을추가하세요

In [2]:
import pandas as pd
import numpy as np

selloutData = \
    pd.read_csv("../dataset/kopo_channel_seasonality_new.csv",encoding = "ms949")

selloutData.loc[ selloutData.QTY < 0]  # QTY가 0보다 작을 때를 조회! (loc이 조회 하는것)

selloutData["QTY_NEW"] = \
    np.where( selloutData.QTY < 0, 0, selloutData.QTY )

selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0
...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0
124654,A10,PRODUCT56,201630,275.0,275.0
124655,A10,PRODUCT61,201630,0.0,0.0
124656,A10,PRODUCT12,201630,15021.0,15021.0


# 3-2 [데이터통합]
### selloutData 자료에서YEAR, WEEK 컬럼을 생성하고WEEK가 52이하인 데이터만 조회한 후 refinedSelloutData 변수에 담으세요

In [3]:
#selloutData    # selloutData 선언(가지고오기)

selloutData = selloutData.astype ({ "REGIONID":str,
                                    "PRODUCT":str,
                                    "YEARWEEK":str,
                                    "QTY":float })
# string타입이여야 분리 가능하니까 YEARWEEK를 str로 바꾼다~!

selloutData.dtypes # str로 잘 바뀌었는지 확인해보는 것

selloutData["YEAR"] = selloutData.YEARWEEK.str[0:4]  # YEARWEEK에서 YEAR만 추출

selloutData["WEEK"] = selloutData.YEARWEEK.str[4:6]  # YEARWEEK에서 WEEK만 추출


selloutData["WEEK"] = selloutData["WEEK"].astype(int)  # 52주차 이하인 데이터를 조회하기 위해서 댜시 int타입으로 변환

selloutData.dtypes # int로 잘 바뀌었는지 확인해보는 것

refinedSelloutData = selloutData.loc[selloutData.WEEK <= 52] #52주차 이하인 것만 loc으로 조회하기  (loc -> db에서 distinct와 같다)

# 검산해보기
#refinedSelloutData.loc[refinedSelloutData.WEEK >= 53] #53주차 이상일때 아무것도 안나오면 옳게 된 것~!

refinedSelloutData 

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,2
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,2
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,2
3,A60,PRODUCT47,201402,0.0,0.0,2014,2
4,A60,PRODUCT56,201402,23.0,23.0,2014,2
...,...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,824.0,2016,30
124654,A10,PRODUCT56,201630,275.0,275.0,2016,30
124655,A10,PRODUCT61,201630,0.0,0.0,2016,30
124656,A10,PRODUCT12,201630,15021.0,15021.0,2016,30


# 4. GROUP BY 집계함수

### 4-1. refinedSelloutData에서→ 지역, 상품, 연주차 컬럼순으로 오름차순정렬하여 sortedData변수에 담으세요 (sort사용)

In [4]:
refinedSelloutData

sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

sortedData = refinedSelloutData.sort_values(by=sortKey,ascending = [True,True,True],ignore_index=True) # true -> 오름차순
                                                                                                       # false -> 내림차순
                                                                                                       # 기본값은 오름차순인데 써주는게 좋다~!
#sortedData

# 4-2. [지역, 상품, 연도별집계]
### sortedData 에서 지역, 상품,연도 단위판매량(QTY_NEW) 의 평균연산 후 groupData 변수에 담으세요 이후 컬럼명을 QTY_MEAN로 변경하세요
### -> 판매량(qty_new)의 평균 연산 (group by 사용)

In [5]:
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A00,PRODUCT34,201401,661.0,661.0,2014,1
1,A00,PRODUCT34,201402,679.0,679.0,2014,2
2,A00,PRODUCT34,201403,578.0,578.0,2014,3
3,A00,PRODUCT34,201404,532.0,532.0,2014,4
4,A00,PRODUCT34,201405,516.0,516.0,2014,5
...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51


In [6]:
groupKey = ["REGIONID","PRODUCT","YEAR"] # 평균값을 구하고 싶은 컬럼

In [ ]:
# 질문 1 

In [7]:
groupData = sortedData.groupby(by=groupKey)["QTY_NEW"].agg(["mean"]).reset_index()
## reset_index()하면 새로운 인덱스를 만들어줌(즉 인덱스 리셋!)

In [8]:
groupData

,REGIONID,PRODUCT,YEAR,mean
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769


In [9]:
groupData.rename(columns = {"mean":"QTY_MEAN"}, inplace=True) # 컬럼이름 변경하는 공식 QTY_NEW -> QTY_MEAN으로~!
                                                                 # inplace = True -> 정렬되어서 정렬된 값이 바로 groupData에 적용됨! (따로 변수 설정 할 필요 없음)

In [10]:
groupData

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769


In [ ]:
# 질문 2 

In [22]:
groupData=sortedData.groupby(groupKey)["QTY_NEW"].agg(["mean"])

In [23]:
groupData

mean
REGIONID PRODUCT   YEAR             
A00      PRODUCT34 2014   275.961538
                   2015    86.634615
                   2016    36.576923
         PRODUCT58 2014     2.673077
                   2015     5.711538
...                              ...
A77      PRODUCT1  2015  3030.019231
                   2016  3375.326923
         PRODUCT12 2014  2035.788462
                   2015  3540.980769
                   2016  4837.153846

[2382 rows x 1 columns]

In [24]:
groupData2=groupData.reset_index()

In [25]:
groupData2

,REGIONID,PRODUCT,YEAR,mean
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769


## 4-3. 이동평균산출 (이동집계함수)

In [11]:
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK
0,A00,PRODUCT34,201401,661.0,661.0,2014,1
1,A00,PRODUCT34,201402,679.0,679.0,2014,2
2,A00,PRODUCT34,201403,578.0,578.0,2014,3
3,A00,PRODUCT34,201404,532.0,532.0,2014,4
4,A00,PRODUCT34,201405,516.0,516.0,2014,5
...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51


In [332]:

sortedData["MA5"] = sortedData.QTY.rolling(window=3, 
                       #min_periods=1, #(최소구간)
                       center=True).mean()  

In [333]:
sortedData["MA5_BASIC"] = sortedData.QTY.rolling(window=3, 
                       #min_periods=1, #(최소구간)
                       center=True).mean()

In [357]:
sortedData["MA5_BASIC_NO"] = sortedData.QTY.rolling(window=3, 
                       #min_periods=1, #(최소구간)
                       center=False).mean()   # false는 자신이 제일 마지막 값 - > 위로만 범위펼침, 여기선 위로 2
                                              # true는 본인이 중심으로(자신도 포함!) 위 아래 범위만큼, 여기선 위아래 1 

In [12]:
## 값이 1개라도 있을때, 현재 값을 중심값으로 이동평균 산출
sortedData["MA5"]=sortedData.QTY.\
            rolling(window=5, 
                     min_periods=1,        # min_periods#(최소구간) -> 값이 1개라도 있을 때
                    center=True).mean()    # true는 본인이 중심으로(자신도 포함!) 위 아래 범위만큼, 여기선 위아래 2
                                           # false는 자신이 제일 마지막 값 - > 위로만 범위펼침, 여기선 위로 4
                                           

## 값이 없으면 계산 x, 현재 값을 중심값으로 이동평균 산출
sortedData["MA5_BASIC"]=sortedData.QTY.\
            rolling(window=5,                    #  min_periods 없어서 값이 없으면 계산 노 
                    center=True).mean()

## 값이 없으면 계산 x, 현재 값을 마지막 값으로 상위 4개와 이동평균 산출
sortedData["MA5_BASIC_NO"]=sortedData.QTY.\
            rolling(window=5,                   
                    center=False).mean()         # false -> 현재 값이 마지막으로 상위값을 이동평균 산출

In [13]:
sortedData

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,MA5,MA5_BASIC,MA5_BASIC_NO
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,639.333333,NaN,NaN
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,612.500000,NaN,NaN
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,593.200000,593.2,NaN
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,545.600000,545.6,NaN
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,491.200000,491.2,593.2
...,...,...,...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4582.600000,4582.6,4119.2
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4718.200000,4718.2,4314.6
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,5513.600000,5513.6,4582.6
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,5854.000000,NaN,4718.2


In [14]:
sortedData2=sortedData.drop(columns=["MA5_BASIC_NO","MA5_BASIC"])

In [15]:
sortedData2

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,MA5
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,639.333333
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,612.500000
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,593.200000
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,545.600000
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,491.200000
...,...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4582.600000
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4718.200000
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,5513.600000
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,5854.000000


In [22]:
# np. where 로 처리???
sortedData2["EFFECT"] = sortedData2.QTY / sortedData2.MA5

In [23]:
sortedData2

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,YEAR,WEEK,MA5,EFFECT
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,639.333333,1.033889
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,612.500000,1.108571
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,593.200000,0.974376
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,545.600000,0.975073
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,491.200000,1.050489
...,...,...,...,...,...,...,...,...,...
123859,A77,PRODUCT12,201648,4152.0,4152.0,2016,48,4582.600000,0.906036
123860,A77,PRODUCT12,201649,5086.0,5086.0,2016,49,4718.200000,1.077953
123861,A77,PRODUCT12,201650,5846.0,5846.0,2016,50,5513.600000,1.060287
123862,A77,PRODUCT12,201651,4933.0,4933.0,2016,51,5854.000000,0.842672


In [24]:
sortedData2.YEAR.drop_duplicates() #db에서 distinct와 같다

0      2014
52     2015
104    2016
Name: YEAR, dtype: object

In [25]:
groupKey = ["REGIONID", "PRODUCT", "WEEK"]

In [26]:
# 3개년도의 지역, 상품, 주차별 평균 EFFECT를 구하고 싶다~! -> group by이용
ratioData=sortedData.groupby(groupKey)["EFFECT"].agg("mean")

In [27]:
ratioData

REGIONID  PRODUCT    WEEK
A00       PRODUCT34  1       0.767107
                     2       0.872595
                     3       0.929402
                     4       1.161136
                     5       0.868462
                               ...   
A77       PRODUCT12  48      1.087273
                     49      0.908397
                     50      1.063923
                     51      0.875609
                     52      0.968998
Name: EFFECT, Length: 41288, dtype: float64

In [47]:
sortedData.to_csv("d:/middleresult.csv",index=False, encoding="ms949") # 중간본 csv파일로 저장

In [ ]:
ratioData.to_csv("d:/finalresult.csv",index=False, encoding="ms949") # 최종본 csv파일로 저장